# Entrenamiento de un SAE sobre las salidad y activaciones de la MLP intermedia de llama3.2 1B

## Costo de entrenamiento

Generalmente, el costo computacional de un LLM está dominado por la evaluación
de sus MLPs (referencia a el seminario en una universidad de un ex empleado
de anthropic)

Una aprocimación simple del costo de entrenamiento de un modelo es

$$
  6ND 
$$
esto en términos de FLOPs (operaciones de punto flotante).
(citar a chinchilla scaling laws)

donde $N$ es el número de parámetros y $D$ la cantidad de muestras en el
conjunto de entrenamiento.

Esto se debe a que, generalmente, cada parámetro actua en una multiplicaciónl
y en una suma de punto flotante, dandonos un costo de $2ND$ tan solo en el
forward pass. Tipicamente, el costo de el backward pass es el doble del forward
pass, haciendo que su costo sea $4ND$. Sumando tenemos el resultado previamente
mencionado.

Sea $N_l$ el número de parámetros de llama.
Como nosotros vamos a entrenar un autoencoder sobre un MLP a la mitad de llama,
solo necesitamos evaluar esa primera mitad. Además, no correremos el backward
pass sobre los parámetros de llama, pues no buscamos modificarlos, es decir, los
mantendrémos fijos. Por esto, tenemos que el FLOPs realizados por tal mitad del
modelo llama es

$$
  N_l D
$$

En cuanto al SAE, solo considerando el costo de aplicar sus matrices, tenemos

$$
  6 (2d_\text{in}d_\text{sae})D
$$

En el caso de gemmascope, entre todos los SAEs que entrenaron, los más pequeños
entrenados en las salidas de las MLPs, se entrenaron en 4 mil millones de
vectores de activaciones, con la dimencion de los vectores en el stream
recidual (y por lo tanto, de la salida de las capas MLP), es 2048, con
$d_\text{sae} = 2028 * 8$.

Deseamos encontrar hiperparámetros para entrenar SAEs, para esto:
- Usamos una primera aproximación razonable modificando los hiperparámetros para
  los autoencoders más pequeños entrenados en salidas de las MLPs de gemma 2
  2 B
- Ajustamos una power law en base a 2 entrenamientos de SAEs más pequeñas,
  usando el mismo learning rate.
- Ajustamos una power law para el learning rate con los hiperparámetos optimos
  que estimó el paso anterior.

Si ignoramos la relación posicional de los tokens y asumimos una distribución
uniforme, tenemos que la entropía por token es

$$
  \log_2 (\text{tamaño del vocabulario})
$$
ya que el vocabulario de gemma2 2B es 256000 y el de llama es 128000, obtenemos
que el número de tokens equivalente sería 4.2 mil millones. Ya que nuestro
modelo es la mitad de tamaño de gemma2 2B, una primera cantidad de datos
razonable para entrenar nuesto sae más grande es $2.1B$

En el caso de llama3.2 1B, eso resultaría en

$$
  N_l D = (2.1 \times 10^9)(1.2 \times 10^9) \approx 2.5 \times 10^{18}
$$
Una RTX 4090 puede realisar cada segundo un máximo de $165 \times 10^{12}$
operaciones con tensores de 16 bits y acumulador de 32 bits (referencia
al reporte técnico v1.0.1), luego, estimamos 4.2 horas de entrenamiento
tan solo considerando la computación en el modelo llama.

Ahora, para estimar las horas-RTX4090 para el autoencoder, en el caso de
entrenarlo en la salida de la MLP intermedia, tendríamos

$$
    6(2.1 \times 10^9)(2048^2)(8)(2) = 8.5 \times 10^{17}
$$

# El modelo

In [16]:
%pip install torch jaxtyping


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import torch
from torch import nn
import jaxtyping
import dataclasses
from torch.nn import functional as F

KeyboardInterrupt: 

In [ ]:
class JumpReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, threshold):
        ctx.save_for_backward(x, threshold)
        return torch.where(x > threshold, x, 0)
    
    @staticmethod
    def backward(ctx, grad_output):
        bandwidth = 0.001
        x, threshold = ctx.saved_tensors
        grad_x = torch.where(x > threshold,
            torch.ones_like(x), torch.zeros_like(x))
        grad_threshold = torch.where(
            abs(x - threshold) < bandwidth/2,
            - threshold/bandwidth, 0)

        return grad_x * grad_output, grad_threshold * grad_output


In [ ]:
class Step(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, threshold):
        ctx.save_for_backward(x, threshold)
        return (x > threshold).to(x.dtype)

    @staticmethod
    def backward(ctx, grad_output):
        bandwidth = 0.001
        x, threshold = ctx.saved_tensors

        grad_threshold = torch.where(
            abs(F.relu(x) - threshold) < bandwidth/2,
            -1.0/bandwidth, 0)
        
        return torch.zeros_like(x), grad_threshold * grad_output

In [ ]:
class Sae(nn.Module):
    def __init__(self, d_in, d_sae, use_pre_enc_bias=True, **kwargs):
        super().__init__(**kwargs)
        self.enc = nn.Linear(d_in, d_sae, dtype=torch.float16)
        self.dec = nn.Linear(d_sae, d_in, dtype=torch.float16)
        with torch.no_grad():
            # normalize each of the d_sae dictonary vectors
            self.dec.weight /= self.dec.weight.norm(dim=0, keepdim=True)
        self.enc.weight = self.dec.weight.clone().t()
        self.enc.bias = torch.zeros_like(self.enc.bias)
        self.dec.bias = torch.zeros_like(self.dec.bias)
        self.log_threshold = nn.Parameter(
            torch.log(torch.full((d_sae,), 0.001, dtype=torch.float16)))
        self.use_pre_enc_bias = use_pre_enc_bias
        def project_out_parallel_grad(dim, tensor):
            @torch.no_grad
            def hook(grad_in):
                # norm along dim=dim of the tensor is assumed to be 1 as we
                # are going to normalize it after every grad update
                dot = (tensor * grad_in).sum(dim=dim, keepdim=True)
                return grad_in - dot * tensor
            return hook

        self.dec.weight.register_hook(
            project_out_parallel_grad(0, self.dec.weight))
                

    def forward(self,
        x,
        return_mask=False,
        return_reconstruction=False,
        return_l0=True,
        return_reconstruction_loss=True,
    ):
        "We compute this much here so that compile() can do its magic"
        # as per train_gpt2.py on karpathy's llm.c repo, there are performance
        # reasons not to return stuff
        d = {}
        original_input = x
        if self.use_pre_enc_bias:
            x = x - self.dec.bias
        
        x = self.enc(x)
        threshold = torch.exp(self.log_threshold)
        s = Step.apply(x, threshold)
        if return_mask:
            d['mask'] = s
        if return_l0:
            d['l0'] = s.sum(-1).mean()
        x = x*s
        x = self.dec(x)
        if return_reconstruction:
            d['reconstruction'] = ((x - original_input)**2).sum(-1).mean()

        return d

In [ ]:
def cosine_schedule_with_warmup(
    current_step: int,
    warmup_steps: int,
    total_steps: int
    ):
    return ...

In [ ]:
steps = 2**16
max_lr = 7e-5
model = Sae(2048, 2048*4)
optimizer = torch.optim.Adam(model.parameters(), lr=max_lr)

scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer,
    lr_lambda=lambda step: cosine_schedule_with_warmup(step, warmup_steps, total_tr)
)
for batch in tqdm(train_loader):
    optimizer.zero_grad()
    d = model(batch)
    reconstruction_loss, l0 = d['reconstuction_loss'], d['l0']
    loss = reconstruction_loss + sparsity_coefficient * l0
    # log losses, compute stats, etc
    loss.backward()
    optimizer.step()
    # TODO: sparsity_coefficient scheduler
    scheduler.step()

    # normalize
    with torch.no_grad():
        model.dec.weight /= model.dec.weight.norm(dim=0, keepdim=True)



In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Interpretación de latentes 

In [7]:
from dotenv import load_dotenv
import os
import openai

load_dotenv()

openai.api_key = os.getenv("API_KEY")


In [ ]:

def fetch_activating_examples_gpt(n) -> list[tuple[int, list[str]]]:
    

    system_prompt =(
        "You are a helpful assistant that writes natural, fluent, anad varied sentences related to the concept of communication."
        "Each sentence must contain exactly one conceptually significant token related to communication, wrapped in doublle angle brackets like this: <<talk>>. "
    )
    user_prompt = (
        f"Generate {n} short sentences (10 to 15 words each) about communication. "
        "In each sentence, mark exactly one key word per sentence — related to communication — by wrapping it with double angle brackets like this: <<talk>>. "
        "Do not explain or list anything, just return the marked sentences as plain text, one per line."
    )

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=300,
        temperature=0.7
    )

    raw_sentences = response.choices[0].message.content.strip().split("\n")

    parsed = []
    for i, sentence in enumerate(raw_sentences):
        tokens = sentence.strip().split()
        marked_idx = None
        for idx, tok in enumerate(tokens):
            if tok.startswith("<<") and tok.endswith(">>"):
                marked_idx = idx
                tokens[idx] = f"<<{tok.strip('<>')}>>"
                break
        
        if marked_idx is not None:
            parsed.append((i,tokens))

    return parsed

In [27]:
def count_gpt_tokens(messages: list[dict], model: str = "gpt-4o") -> int:
    try:
        enc = tiktoken.encoding_for_model(model)
    except KeyError:
        enc = tiktoken.get_encoding("cl100k_base")  # fallback
    # Reglas de tokenización para ChatML
    tokens_per_message = 3  # <im_start>{role}\n{content}<im_end>\n
    tokens_per_name = 1     # if name is present
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(enc.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # for priming <|start|>assistant
    return num_tokens

In [28]:
def create_prompt_gpt4o_from_simulated(
        examples: list[tuple[int, list[str]]], 
        use_chain_of_thought=True) -> dict[str, str]:
    """
    Construye un prompt de GPT-4o a partir de ejemplos simulados con tokens activadores marcados.
    """
    formatted = "\n".join(f"{i+1}. {' '.join(tokens)}" for i, tokens in examples)

    system_prompt = (
       "You are GPT-4o, an advanced AI model analyzing the behavior of a latent neuron in a transformer-based language model.\n"
        "You are given a list of activation examples in <examples></examples>. Each example is a sentence where a single token has been marked "
        "with double angle brackets (<<token>>) to indicate the token that triggered the strongest activation of this latent.\n"
        "Your goal is to reverse-engineer what this neuron is detecting, based on the marked tokens across the examples.\n"
        "You must generalize a concept that explains why those tokens might consistently activate this neuron.\n"
        "The interpretation must be short (≤20 words), free of formatting, and semantically precise. Avoid generic answers or listing multiple unrelated ideas."
    )

    user_prompt = (
        "<examples>\n"
        f"{formatted}\n"
        "</examples>\n\n"
        "Now, identify what kind of concept or feature causes this latent neuron to activate."
    )

    if use_chain_of_thought:
        assistant_prompt = (
            "First, analyze the examples. Group them if useful. Consider the roles, context, and semantics of the marked tokens.\n"
            "Then, provide a clear and concise interpretation — in 20 words or fewer — describing what this neuron detects.\n"
            "Final output starts with: This neuron activates on"
    )
    else: 
        assistant_prompt = "This neuron activates on"

    return {
        "system": system_prompt,
        "user": f"The activating examples are:\n\n{formatted}",
        "assistant": assistant_prompt,
    }


In [29]:
def build_batched_prompt_with_token_estimate(
    examples: list[tuple[int, list[str]]], 
    batch_size: int = 10,
    use_chain_of_thought: bool = True,
    model: str = "gpt-4o",
    token_limit: int = 500
) -> list[dict]:
    """
    Divide los ejemplos en lotes y crea prompts con estimación de tokens para cada lote.
    
    Args:
        examples: Lista de tuplas (índice, tokens) con ejemplos marcados
        batch_size: Número de ejemplos por lote
        use_chain_of_thought: Si usar cadena de pensamiento en el prompt
        model: Modelo para estimación de tokens
        token_limit: Límite de tokens para marcar advertencia
    
    Returns:
        Lista de diccionarios con información de cada lote
    """
    batched_prompts = []
    
    # Dividir ejemplos en lotes
    for i in range(0, len(examples), batch_size):
        batch_examples = examples[i:i + batch_size]
        
        # Crear prompt para este lote
        prompt = create_prompt_gpt4o_from_simulated(
            batch_examples, 
            use_chain_of_thought=use_chain_of_thought
        )
        
        # Convertir prompt a formato de mensajes para contar tokens
        messages = [
            {"role": "system", "content": prompt["system"]},
            {"role": "user", "content": prompt["user"]},
            {"role": "assistant", "content": prompt["assistant"]},
        ]
        
        # Contar tokens
        n_tokens = count_gpt_tokens(messages, model=model)
        
        # Crear entrada del lote
        batch_info = {
            "prompt": prompt,
            "examples": batch_examples,
            "n_tokens": n_tokens,
            "over_limit": n_tokens > token_limit,
            "batch_size": len(batch_examples)
        }
        
        batched_prompts.append(batch_info)
    
    return batched_prompts

In [30]:
simulated_examples = fetch_activating_examples_gpt(n=30)
batched_prompts = build_batched_prompt_with_token_estimate(
    simulated_examples,
    batch_size=10,  # Puedes ajustar el tamaño
    use_chain_of_thought=True,
    model="gpt-4o",
    token_limit=500  # Umbral opcional para advertir
)

# Mostrar estadísticas de cada lote
for i, batch in enumerate(batched_prompts):
    print(f"Lote {i+1}: {batch['n_tokens']} tokens. ¿Sobrepasa el límite? {batch['over_limit']}")


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:

def count_gpt_tokens(messages: list[dict], model: str = "gpt-4o") -> int:
    try:
        enc = tiktoken.encoding_for_model(model)
    except KeyError:
        enc = tiktoken.get_encoding("cl100k_base")  # fallback

    # Reglas de tokenización para ChatML
    tokens_per_message = 3  # <im_start>{role}\n{content}<im_end>\n
    tokens_per_name = 1     # if name is present
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(enc.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # for priming <|start|>assistant
    return num_tokens


In [ ]:
import openai
def get_gpt4o_explanation_from_prompt(prompts: dict, n_completions=5, max_tokens=100, verbose = True) -> list[str]:
    """
    Llama a la API de GPT-4o con un prompt ya generado y devuelve las interpretaciones.
    """

    messages = [
        {"role": "system", "content": prompts["system"]},
        {"role": "user", "content": prompts["user"]},
        {"role": "assistant", "content": prompts["assistant"]},
    ]

    prompt_tokens = count_gpt_tokens(messages, model="gpt-4o")
    if verbose:
        print(f"Prompt length: {prompt_tokens} tokens")
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": prompts["system"]},
            {"role": "user", "content": prompts["user"]},
            {"role": "assistant", "content": prompts["assistant"]},
        ],
        n=n_completions,
        max_tokens=max_tokens,
        temperature=0.7,
    )
    return [choice.message.content.strip() for choice in response.choices]


In [ ]:
for i, batch in enumerate(batched_prompts):
    print(f"\n--- Lote {i+1} ---")
    respuestas = get_gpt4o_explanation_from_prompt(batch["prompt"], n_completions=2)
    for r in respuestas:
        print(r)


--- Lote 1 ---
Prompt length: 395 tokens


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


enviar los datos en grupo batching para obtener descuento en el uso de la API